# 7. Transformer变体和现代应用

在完成了完整Transformer的学习后，让我们探索Transformer家族的重要变体和现代应用。这些变体推动了NLP和其他领域的重大突破。

## 7.1 Transformer家族概览

自2017年原始Transformer发布以来，出现了众多重要的变体：

### 按架构分类：
1. **仅编码器架构**：BERT、RoBERTa、DeBERTa等
2. **仅解码器架构**：GPT系列、PaLM、LLaMA等
3. **编码器-解码器架构**：T5、BART、mT5等

### 按应用领域分类：
1. **自然语言处理**：BERT、GPT、T5等
2. **计算机视觉**：Vision Transformer (ViT)、DETR等
3. **多模态**：CLIP、DALLE、GPT-4等
4. **科学计算**：AlphaFold、ESM等

![Transformer家族树](images/transformer_family.png)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from typing import Optional, Tuple, List

# 从utils导入基础组件
from utils import (
    MultiHeadAttention, 
    SinusoidalPositionalEncoding, 
    FeedForwardNetwork,
    TransformerBlock,
    PreLNTransformerBlock,
    create_padding_mask,
    create_causal_mask,
    setup_matplotlib_chinese,
    set_random_seed
)

# 设置环境
setup_matplotlib_chinese()
set_random_seed(42)

print(f"PyTorch版本: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

## 7.2 BERT：双向编码器表示

BERT (Bidirectional Encoder Representations from Transformers) 是最具影响力的Transformer变体之一：

### 核心特点：
- **仅编码器架构**：只使用Transformer的编码器部分
- **双向上下文**：能够同时看到左边和右边的上下文
- **预训练+微调**：在大规模语料上预训练，然后在具体任务上微调

### 预训练任务：
1. **掩码语言模型（MLM）**：随机掩盖一些词，让模型预测
2. **下一句预测（NSP）**：预测两个句子是否连续

In [ ]:
class BERTModel(nn.Module):
    """
    简化的BERT模型实现
    """
    def __init__(self, vocab_size: int, d_model: int = 768, num_heads: int = 12, 
                 d_ff: int = 3072, num_layers: int = 12, max_seq_len: int = 512, 
                 dropout: float = 0.1):
        super(BERTModel, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = vocab_size
        
        # 嵌入层
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_seq_len, d_model)
        self.token_type_embedding = nn.Embedding(2, d_model)  # 用于区分句子A和B
        
        # Transformer编码器层
        self.layers = nn.ModuleList([
            PreLNTransformerBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # 输出层
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
        # 用于MLM和NSP的头
        self.mlm_head = nn.Linear(d_model, vocab_size)
        self.nsp_head = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.Tanh(),
            nn.Linear(d_model, 2)
        )
        
        # 特殊token索引
        self.cls_token_id = 0  # [CLS]
        self.sep_token_id = 1  # [SEP]
        self.mask_token_id = 2  # [MASK]
        self.pad_token_id = 3  # [PAD]
        
    def forward(self, input_ids: torch.Tensor, token_type_ids: Optional[torch.Tensor] = None,
                attention_mask: Optional[torch.Tensor] = None):
        """
        前向传播
        
        Args:
            input_ids: [batch_size, seq_len]
            token_type_ids: [batch_size, seq_len] 句子类型ID
            attention_mask: [batch_size, seq_len] 注意力掩码
        
        Returns:
            last_hidden_state: [batch_size, seq_len, d_model]
            pooler_output: [batch_size, d_model] 用于分类的[CLS]表示
        """
        batch_size, seq_len = input_ids.size()
        
        # 创建位置ID
        position_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0).expand(batch_size, -1)
        
        # 如果没有提供token_type_ids，默认都是0
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        
        # 嵌入
        token_embeds = self.token_embedding(input_ids)
        position_embeds = self.position_embedding(position_ids)
        token_type_embeds = self.token_type_embedding(token_type_ids)
        
        # 合并嵌入
        embeddings = token_embeds + position_embeds + token_type_embeds
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        
        # 创建注意力掩码
        if attention_mask is not None:
            # 扩展掩码维度: [batch_size, 1, 1, seq_len]
            extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
            extended_attention_mask = extended_attention_mask.expand(-1, -1, seq_len, -1)
        else:
            extended_attention_mask = None
        
        # 通过Transformer层
        hidden_states = embeddings
        attention_weights_list = []
        
        for layer in self.layers:
            hidden_states, attention_weights = layer(hidden_states, extended_attention_mask)
            attention_weights_list.append(attention_weights)
        
        # 获取[CLS]的表示用于句子级任务
        pooler_output = self.nsp_head(hidden_states[:, 0, :])  # [CLS]是第一个token
        
        return {
            'last_hidden_state': hidden_states,
            'pooler_output': pooler_output,
            'attention_weights': attention_weights_list
        }
    
    def get_mlm_predictions(self, hidden_states: torch.Tensor):
        """获取掩码语言模型的预测"""
        return self.mlm_head(hidden_states)

# 创建BERT模型
bert_model = BERTModel(
    vocab_size=1000,
    d_model=256,  # 使用较小的维度用于演示
    num_heads=8,
    d_ff=1024,
    num_layers=4,
    max_seq_len=128
)

# 测试BERT模型
batch_size = 2
seq_len = 16

# 模拟输入
input_ids = torch.randint(4, 1000, (batch_size, seq_len))  # 避开特殊token
input_ids[:, 0] = 0  # 设置[CLS]
input_ids[:, seq_len//2] = 1  # 设置[SEP]

attention_mask = torch.ones(batch_size, seq_len)
token_type_ids = torch.zeros(batch_size, seq_len)
token_type_ids[:, seq_len//2+1:] = 1  # 第二个句子

# 前向传播
outputs = bert_model(input_ids, token_type_ids, attention_mask)

print(f"BERT模型输出:")
print(f"最后隐藏层: {outputs['last_hidden_state'].shape}")
print(f"池化输出: {outputs['pooler_output'].shape}")
print(f"注意力层数: {len(outputs['attention_weights'])}")
print(f"模型参数量: {sum(p.numel() for p in bert_model.parameters()):,}")

## 7.3 GPT：生成式预训练Transformer

GPT (Generative Pre-trained Transformer) 系列是另一个重要的Transformer变体：

### 核心特点：
- **仅解码器架构**：使用因果掩码的自注意力
- **自回归生成**：从左到右逐个生成token
- **语言建模**：预测下一个词的概率分布

### GPT系列演进：
- **GPT-1** (117M参数)：证明了预训练+微调的有效性
- **GPT-2** (1.5B参数)：展示了规模化的威力
- **GPT-3** (175B参数)：实现了Few-shot Learning
- **GPT-4** (未公布参数)：多模态能力，更强的推理

In [ ]:
class GPTModel(nn.Module):
    """
    简化的GPT模型实现
    """
    def __init__(self, vocab_size: int, d_model: int = 768, num_heads: int = 12, 
                 d_ff: int = 3072, num_layers: int = 12, max_seq_len: int = 1024, 
                 dropout: float = 0.1):
        super(GPTModel, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len
        
        # 嵌入层
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_seq_len, d_model)
        
        # Transformer解码器层（使用因果掩码）
        self.layers = nn.ModuleList([
            PreLNTransformerBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # 输出层
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
        # 语言建模头
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        
        # 权重共享（可选）
        # self.lm_head.weight = self.token_embedding.weight
        
    def forward(self, input_ids: torch.Tensor, attention_mask: Optional[torch.Tensor] = None):
        """
        前向传播
        
        Args:
            input_ids: [batch_size, seq_len]
            attention_mask: [batch_size, seq_len]
        
        Returns:
            logits: [batch_size, seq_len, vocab_size]
            hidden_states: [batch_size, seq_len, d_model]
        """
        batch_size, seq_len = input_ids.size()
        
        # 创建位置ID
        position_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0).expand(batch_size, -1)
        
        # 嵌入
        token_embeds = self.token_embedding(input_ids)
        position_embeds = self.position_embedding(position_ids)
        
        hidden_states = token_embeds + position_embeds
        hidden_states = self.dropout(hidden_states)
        
        # 创建因果掩码
        causal_mask = create_causal_mask(seq_len).to(input_ids.device)
        causal_mask = causal_mask.unsqueeze(0).expand(batch_size, -1, -1)
        
        # 结合填充掩码（如果提供）
        if attention_mask is not None:
            padding_mask = attention_mask.unsqueeze(1).expand(-1, seq_len, -1)
            combined_mask = causal_mask & padding_mask
        else:
            combined_mask = causal_mask
        
        # 通过Transformer层
        attention_weights_list = []
        
        for layer in self.layers:
            hidden_states, attention_weights = layer(hidden_states, combined_mask)
            attention_weights_list.append(attention_weights)
        
        hidden_states = self.layer_norm(hidden_states)
        
        # 语言建模输出
        logits = self.lm_head(hidden_states)
        
        return {
            'logits': logits,
            'hidden_states': hidden_states,
            'attention_weights': attention_weights_list
        }
    
    def generate(self, input_ids: torch.Tensor, max_new_tokens: int = 50, 
                 temperature: float = 1.0, top_k: int = None, top_p: float = None):
        """
        文本生成方法
        
        Args:
            input_ids: 输入的token序列
            max_new_tokens: 最大生成的新token数
            temperature: 温度参数，控制随机性
            top_k: Top-K采样
            top_p: Top-P采样（核采样）
        
        Returns:
            generated_ids: 生成的完整序列
        """
        self.eval()
        generated_ids = input_ids.clone()
        
        with torch.no_grad():
            for _ in range(max_new_tokens):
                # 如果序列太长，截断
                if generated_ids.size(1) > self.max_seq_len:
                    generated_ids = generated_ids[:, -self.max_seq_len:]
                
                # 前向传播
                outputs = self.forward(generated_ids)
                logits = outputs['logits'][:, -1, :] / temperature  # 取最后一个位置的logits
                
                # 采样策略
                if top_k is not None:
                    # Top-K采样
                    top_k_logits, top_k_indices = torch.topk(logits, top_k)
                    logits = torch.full_like(logits, float('-inf'))
                    logits.scatter_(1, top_k_indices, top_k_logits)
                
                if top_p is not None:
                    # Top-P采样（核采样）
                    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
                    
                    # 移除累积概率超过top_p的token
                    sorted_indices_to_remove = cumulative_probs > top_p
                    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
                    sorted_indices_to_remove[..., 0] = 0
                    
                    indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
                    logits[indices_to_remove] = float('-inf')
                
                # 计算概率并采样
                probs = F.softmax(logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
                
                # 添加到生成序列
                generated_ids = torch.cat([generated_ids, next_token], dim=1)
        
        return generated_ids

# 创建GPT模型
gpt_model = GPTModel(
    vocab_size=1000,
    d_model=256,  # 使用较小的维度用于演示
    num_heads=8,
    d_ff=1024,
    num_layers=4,
    max_seq_len=128
)

# 测试GPT模型
input_ids = torch.randint(0, 1000, (1, 10))
outputs = gpt_model(input_ids)

print(f"GPT模型输出:")
print(f"Logits形状: {outputs['logits'].shape}")
print(f"隐藏状态形状: {outputs['hidden_states'].shape}")
print(f"模型参数量: {sum(p.numel() for p in gpt_model.parameters()):,}")

# 测试文本生成
generated = gpt_model.generate(input_ids, max_new_tokens=10, temperature=0.8, top_k=50)
print(f"\n生成序列长度: {generated.shape[1]} (原长度: {input_ids.shape[1]})")
print(f"生成的token: {generated[0].tolist()}")

## 7.4 Vision Transformer (ViT)

Vision Transformer将Transformer成功应用到计算机视觉领域：

### 核心思想：
1. **图像分块**：将图像分割为固定大小的patch
2. **线性嵌入**：将每个patch通过线性层映射为token
3. **位置编码**：为每个patch添加位置信息
4. **Transformer编码**：使用标准的Transformer编码器
5. **分类头**：使用[CLS] token进行图像分类

In [ ]:
class VisionTransformer(nn.Module):
    """
    Vision Transformer (ViT) 实现
    """
    def __init__(self, img_size: int = 224, patch_size: int = 16, num_classes: int = 1000,
                 d_model: int = 768, num_heads: int = 12, d_ff: int = 3072, 
                 num_layers: int = 12, channels: int = 3, dropout: float = 0.1):
        super(VisionTransformer, self).__init__()
        
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.d_model = d_model
        
        # Patch嵌入
        self.patch_embedding = nn.Conv2d(
            channels, d_model, kernel_size=patch_size, stride=patch_size
        )
        
        # 可学习的[CLS] token
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))
        
        # 位置编码
        self.position_embedding = nn.Parameter(
            torch.randn(1, self.num_patches + 1, d_model)
        )
        
        # Transformer编码器
        self.layers = nn.ModuleList([
            PreLNTransformerBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        # 输出层
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
        # 分类头
        self.classification_head = nn.Linear(d_model, num_classes)
        
    def forward(self, x: torch.Tensor):
        """
        前向传播
        
        Args:
            x: [batch_size, channels, height, width]
        
        Returns:
            logits: [batch_size, num_classes]
            attention_weights: 注意力权重列表
        """
        batch_size = x.size(0)
        
        # 1. Patch嵌入
        # [batch_size, d_model, num_patches_h, num_patches_w]
        patch_embeds = self.patch_embedding(x)
        
        # 重塑为序列: [batch_size, num_patches, d_model]
        patch_embeds = patch_embeds.flatten(2).transpose(1, 2)
        
        # 2. 添加[CLS] token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        # [batch_size, num_patches + 1, d_model]
        token_embeds = torch.cat([cls_tokens, patch_embeds], dim=1)
        
        # 3. 添加位置编码
        token_embeds = token_embeds + self.position_embedding
        token_embeds = self.dropout(token_embeds)
        
        # 4. 通过Transformer层
        hidden_states = token_embeds
        attention_weights_list = []
        
        for layer in self.layers:
            hidden_states, attention_weights = layer(hidden_states)
            attention_weights_list.append(attention_weights)
        
        hidden_states = self.layer_norm(hidden_states)
        
        # 5. 分类
        # 使用[CLS] token（第一个位置）进行分类
        cls_output = hidden_states[:, 0, :]
        logits = self.classification_head(cls_output)
        
        return {
            'logits': logits,
            'hidden_states': hidden_states,
            'attention_weights': attention_weights_list
        }

# 创建ViT模型
vit_model = VisionTransformer(
    img_size=224,
    patch_size=16,
    num_classes=10,  # 简化为10类
    d_model=256,     # 使用较小的维度用于演示
    num_heads=8,
    d_ff=1024,
    num_layers=4,
    channels=3
)

# 测试ViT模型
batch_size = 2
test_images = torch.randn(batch_size, 3, 224, 224)

outputs = vit_model(test_images)

print(f"Vision Transformer输出:")
print(f"输入图像形状: {test_images.shape}")
print(f"分类logits形状: {outputs['logits'].shape}")
print(f"隐藏状态形状: {outputs['hidden_states'].shape}")
print(f"Patch数量: {vit_model.num_patches}")
print(f"序列长度: {vit_model.num_patches + 1} (包含CLS token)")
print(f"模型参数量: {sum(p.numel() for p in vit_model.parameters()):,}")

## 7.5 可视化ViT的注意力模式

让我们可视化Vision Transformer学到的注意力模式：

In [ ]:
def visualize_vit_attention(model, image, layer_idx=-1, head_idx=0):
    """
    可视化ViT的注意力模式
    
    Args:
        model: ViT模型
        image: 输入图像 [1, 3, H, W]
        layer_idx: 要可视化的层索引
        head_idx: 要可视化的头索引
    """
    model.eval()
    
    with torch.no_grad():
        outputs = model(image)
        
        # 获取指定层和头的注意力权重
        attention_weights = outputs['attention_weights'][layer_idx]  # [1, num_heads, seq_len, seq_len]
        attention_map = attention_weights[0, head_idx].cpu().numpy()  # [seq_len, seq_len]
        
        # 获取[CLS] token对所有patch的注意力
        cls_attention = attention_map[0, 1:]  # 去掉[CLS] token自己
        
        # 重塑为图像形状
        num_patches_per_side = int(np.sqrt(len(cls_attention)))
        attention_image = cls_attention.reshape(num_patches_per_side, num_patches_per_side)
        
        # 可视化
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # 原始图像
        if image.shape[1] == 3:  # RGB图像
            img_np = image[0].permute(1, 2, 0).cpu().numpy()
            img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())  # 归一化到[0,1]
            axes[0].imshow(img_np)
        else:
            axes[0].imshow(image[0, 0].cpu().numpy(), cmap='gray')
        axes[0].set_title('原始图像')
        axes[0].axis('off')
        
        # [CLS]注意力热力图
        im1 = axes[1].imshow(attention_image, cmap='hot', interpolation='nearest')
        axes[1].set_title(f'[CLS] Token注意力\n(层{layer_idx}, 头{head_idx})')
        axes[1].axis('off')
        plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
        
        # 完整注意力矩阵
        im2 = axes[2].imshow(attention_map, cmap='Blues')
        axes[2].set_title('完整注意力矩阵')
        axes[2].set_xlabel('Key位置')
        axes[2].set_ylabel('Query位置')
        plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)
        
        plt.tight_layout()
        plt.show()
        
        return attention_image, attention_map

# 创建一个简单的测试图像（中心有一个亮块）
test_image = torch.zeros(1, 3, 224, 224)
test_image[:, :, 80:144, 80:144] = 1.0  # 中心亮块
test_image[:, :, 100:124, 100:124] = 0.5  # 中心更亮的小块

# 可视化注意力
attention_img, attention_full = visualize_vit_attention(vit_model, test_image, layer_idx=-1, head_idx=0)

print(f"注意力图像形状: {attention_img.shape}")
print(f"完整注意力矩阵形状: {attention_full.shape}")

## 7.6 效率优化变体

随着Transformer模型规模的增长，出现了许多专注于效率优化的变体：

In [ ]:
class LinearAttention(nn.Module):
    """
    线性注意力机制
    将复杂度从O(n²)降低到O(n)
    """
    def __init__(self, d_model: int, num_heads: int, dropout: float = 0.1):
        super(LinearAttention, self).__init__()
        
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor):
        batch_size, seq_len, _ = query.size()
        
        Q = self.W_q(query).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        
        # 应用特征映射函数（这里使用ReLU作为示例）
        Q = F.relu(Q)
        K = F.relu(K)
        
        # 线性注意力计算: O(n) 复杂度
        # 计算 K^T V
        KV = torch.matmul(K.transpose(-2, -1), V)  # [batch, heads, d_k, d_k]
        
        # 计算 Q (K^T V)
        output = torch.matmul(Q, KV)  # [batch, heads, seq_len, d_k]
        
        # 归一化
        normalizer = torch.matmul(Q, K.sum(dim=-2, keepdim=True).transpose(-2, -1))
        output = output / (normalizer + 1e-6)
        
        # 重塑和输出投影
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        output = self.W_o(output)
        
        return output

class SlidingWindowAttention(nn.Module):
    """
    滑动窗口注意力
    只关注局部窗口内的token，降低复杂度
    """
    def __init__(self, d_model: int, num_heads: int, window_size: int = 128, dropout: float = 0.1):
        super(SlidingWindowAttention, self).__init__()
        
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.window_size = window_size
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor):
        batch_size, seq_len, _ = query.size()
        
        Q = self.W_q(query).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        
        # 创建滑动窗口掩码
        mask = torch.zeros(seq_len, seq_len, device=query.device)
        for i in range(seq_len):
            start = max(0, i - self.window_size // 2)
            end = min(seq_len, i + self.window_size // 2 + 1)
            mask[i, start:end] = 1
        
        # 计算注意力分数
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # 应用滑动窗口掩码
        scores = scores.masked_fill(mask.unsqueeze(0).unsqueeze(0) == 0, -1e9)
        
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        output = torch.matmul(attention_weights, V)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        output = self.W_o(output)
        
        return output

# 比较不同注意力机制的复杂度
def compare_attention_complexity():
    """
    比较不同注意力机制的计算复杂度
    """
    seq_lengths = [128, 256, 512, 1024, 2048]
    d_model = 512
    
    # 计算复杂度（简化，以FLOPs为单位）
    standard_complexity = []
    linear_complexity = []
    sliding_window_complexity = []
    
    window_size = 128
    
    for n in seq_lengths:
        # 标准注意力: O(n² * d)
        standard = n * n * d_model
        
        # 线性注意力: O(n * d²)
        linear = n * d_model * d_model
        
        # 滑动窗口注意力: O(n * w * d), w为窗口大小
        sliding = n * window_size * d_model
        
        standard_complexity.append(standard / 1e9)  # 转换为G FLOPs
        linear_complexity.append(linear / 1e9)
        sliding_window_complexity.append(sliding / 1e9)
    
    # 绘制复杂度对比
    plt.figure(figsize=(12, 8))
    
    plt.plot(seq_lengths, standard_complexity, 'r-o', label='标准注意力 O(n²d)', linewidth=2)
    plt.plot(seq_lengths, linear_complexity, 'g-s', label='线性注意力 O(nd²)', linewidth=2)
    plt.plot(seq_lengths, sliding_window_complexity, 'b-^', label=f'滑动窗口注意力 O(nwd), w={window_size}', linewidth=2)
    
    plt.xlabel('序列长度')
    plt.ylabel('计算复杂度 (G FLOPs)')
    plt.title('不同注意力机制的计算复杂度对比')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    plt.show()
    
    # 打印复杂度表格
    print("不同注意力机制的计算复杂度对比 (G FLOPs):")
    print("=" * 60)
    print(f"{'序列长度':<10} {'标准注意力':<15} {'线性注意力':<15} {'滑动窗口':<15}")
    print("-" * 60)
    
    for i, n in enumerate(seq_lengths):
        print(f"{n:<10} {standard_complexity[i]:<15.2f} {linear_complexity[i]:<15.2f} {sliding_window_complexity[i]:<15.2f}")

# 运行复杂度对比
compare_attention_complexity()

# 测试线性注意力
d_model = 128
num_heads = 4
seq_len = 64
batch_size = 2

test_input = torch.randn(batch_size, seq_len, d_model)

# 标准注意力
standard_attn = MultiHeadAttention(d_model, num_heads)
standard_output, _ = standard_attn(test_input, test_input, test_input)

# 线性注意力
linear_attn = LinearAttention(d_model, num_heads)
linear_output = linear_attn(test_input, test_input, test_input)

# 滑动窗口注意力
sliding_attn = SlidingWindowAttention(d_model, num_heads, window_size=32)
sliding_output = sliding_attn(test_input, test_input, test_input)

print(f"\n各种注意力机制输出形状对比:")
print(f"输入: {test_input.shape}")
print(f"标准注意力输出: {standard_output.shape}")
print(f"线性注意力输出: {linear_output.shape}")
print(f"滑动窗口注意力输出: {sliding_output.shape}")

## 7.7 Transformer应用领域总结

让我们总结Transformer在各个领域的应用：

In [ ]:
import pandas as pd

# 创建Transformer应用领域总结表
transformer_applications = {
    '领域': [
        '自然语言处理', '自然语言处理', '自然语言处理', '自然语言处理',
        '计算机视觉', '计算机视觉', '计算机视觉',
        '多模态', '多模态', '多模态',
        '科学计算', '科学计算', '科学计算',
        '语音处理', '语音处理',
        '推荐系统', '时间序列'
    ],
    '模型名称': [
        'BERT', 'GPT-3/4', 'T5', 'PaLM',
        'ViT', 'DETR', 'Swin Transformer',
        'CLIP', 'DALLE-2', 'GPT-4V',
        'AlphaFold', 'ESM', 'ProtTrans',
        'Whisper', 'Speech-T5',
        'SASRec', 'Informer'
    ],
    '主要任务': [
        '文本理解、分类', '文本生成', '文本到文本生成', '通用语言任务',
        '图像分类', '目标检测', '分层视觉任务',
        '图文匹配', '文本到图像生成', '多模态理解',
        '蛋白质结构预测', '蛋白质序列分析', '蛋白质功能预测',
        '语音识别', '语音合成',
        '序列推荐', '时间序列预测'
    ],
    '关键创新': [
        '双向编码', '自回归生成', '统一编解码', '规模化训练',
        'Patch嵌入', '端到端检测', '分层注意力',
        '对比学习', '扩散模型结合', '视觉语言融合',
        '几何约束', '进化信息利用', '多尺度特征',
        '多语言支持', '统一框架',
        '序列建模', '长序列处理'
    ]
}

df = pd.DataFrame(transformer_applications)

print("Transformer在各领域的应用总结")
print("=" * 80)
print(df.to_string(index=False))

# 绘制应用领域分布饼图
domain_counts = df['领域'].value_counts()

plt.figure(figsize=(10, 8))
colors = ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99', '#FF99CC', '#99CCFF', '#FFB366']
wedges, texts, autotexts = plt.pie(domain_counts.values, labels=domain_counts.index, 
                                  autopct='%1.1f%%', colors=colors, startangle=90)

plt.title('Transformer应用领域分布', fontsize=16, pad=20)

# 美化文字
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.axis('equal')
plt.show()

# 发展时间线
timeline_data = {
    '年份': [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
    '重要模型/事件': [
        'Transformer论文发布',
        'BERT发布，NLP大突破',
        'GPT-2发布，生成能力展现',
        'GPT-3发布，ViT论文，多领域扩展',
        'CLIP、DALLE等多模态模型',
        'ChatGPT发布，AI应用爆发',
        'GPT-4、多模态大模型',
        'Sora、更强的多模态能力'
    ],
    '参数规模': [
        '65M (Base)',
        '340M (BERT-Large)',
        '1.5B (GPT-2)',
        '175B (GPT-3)',
        '12B (CLIP)',
        '175B (ChatGPT)',
        '未公布 (GPT-4)',
        '未公布 (Sora)'
    ]
}

timeline_df = pd.DataFrame(timeline_data)

print("\n\nTransformer发展时间线")
print("=" * 80)
print(timeline_df.to_string(index=False))

# 绘制参数规模发展趋势
param_counts = [65, 340, 1500, 175000, 12000, 175000, 500000, 1000000]  # 估算值，单位M

plt.figure(figsize=(12, 6))
plt.plot(timeline_data['年份'][:6], param_counts[:6], 'bo-', linewidth=3, markersize=8)
plt.yscale('log')
plt.xlabel('年份')
plt.ylabel('参数规模 (百万)')
plt.title('Transformer模型参数规模发展趋势')
plt.grid(True, alpha=0.3)

# 添加标注
annotations = ['Transformer', 'BERT', 'GPT-2', 'GPT-3', 'CLIP', 'ChatGPT']
for i, (year, params, label) in enumerate(zip(timeline_data['年份'][:6], param_counts[:6], annotations)):
    plt.annotate(label, (year, params), textcoords="offset points", 
                xytext=(0,10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

## 7.8 未来发展趋势

让我们讨论Transformer的未来发展方向：

In [ ]:
# 未来发展趋势分析
future_trends = {
    '发展方向': [
        '效率优化',
        '多模态融合',
        '长序列建模',
        '稀疏化技术',
        '架构创新',
        '训练方法',
        '应用拓展',
        '可解释性'
    ],
    '具体技术': [
        '线性注意力、Flash Attention、量化压缩',
        'CLIP升级版、统一多模态架构',
        'Longformer、BigBird、新位置编码',
        'MoE、稀疏注意力、动态计算',
        'RetNet、Mamba、新激活函数',
        'In-context Learning、强化学习优化',
        '科学计算、机器人、游戏AI',
        '注意力可视化、因果分析'
    ],
    '挑战': [
        '维持性能的同时降低计算成本',
        '不同模态的有效对齐和融合',
        '内存限制、位置编码外推',
        '稀疏模式学习、硬件适配',
        '训练稳定性、收敛速度',
        '数据效率、安全对齐',
        '领域适应、实时推理',
        '黑盒问题、决策透明度'
    ],
    '预期影响': [
        '大模型民主化、边缘部署',
        '通用人工智能、创意内容生成',
        '长文档理解、复杂推理',
        '超大规模模型、个性化AI',
        '更强的表达能力、新兴任务',
        '更安全可靠的AI系统',
        '科学发现加速、生产力革命',
        '可信AI、负责任的AI部署'
    ]
}

trends_df = pd.DataFrame(future_trends)

print("Transformer未来发展趋势分析")
print("=" * 100)
print(trends_df.to_string(index=False, max_colwidth=30))

# 技术成熟度曲线
technologies = [
    'GPT系列', 'BERT系列', 'Vision Transformer', '多模态Transformer',
    '效率优化技术', '长序列建模', '稀疏Transformer', '科学计算应用'
]

maturity_scores = [0.9, 0.95, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]  # 成熟度评分
adoption_scores = [0.9, 0.95, 0.7, 0.6, 0.4, 0.3, 0.2, 0.2]  # 采用程度评分

plt.figure(figsize=(12, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(technologies)))

for i, (tech, maturity, adoption, color) in enumerate(zip(technologies, maturity_scores, adoption_scores, colors)):
    plt.scatter(maturity, adoption, s=200, c=[color], alpha=0.7, edgecolors='black')
    plt.annotate(tech, (maturity, adoption), xytext=(5, 5), 
                textcoords='offset points', fontsize=9, 
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

plt.xlabel('技术成熟度', fontsize=12)
plt.ylabel('市场采用程度', fontsize=12)
plt.title('Transformer相关技术成熟度曲线', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim(0, 1)
plt.ylim(0, 1)

# 添加象限标签
plt.text(0.05, 0.95, '新兴技术', fontsize=10, alpha=0.7, 
         bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', alpha=0.5))
plt.text(0.75, 0.95, '成熟技术', fontsize=10, alpha=0.7,
         bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', alpha=0.5))
plt.text(0.05, 0.05, '实验阶段', fontsize=10, alpha=0.7,
         bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', alpha=0.5))
plt.text(0.75, 0.05, '落后技术', fontsize=10, alpha=0.7,
         bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n关键洞察:")
print("1. GPT和BERT系列已经非常成熟，广泛应用")
print("2. Vision Transformer快速发展，正在取代CNN")
print("3. 多模态技术是当前热点，潜力巨大")
print("4. 效率优化和长序列建模是未来重点")
print("5. 稀疏化和科学计算应用仍在早期阶段")

## 总结

在这个最后的教程中，我们全面探索了Transformer的变体和应用：

### 🏗️ 主要架构变体：
1. **BERT系列**：双向编码器，理解任务的标杆
2. **GPT系列**：自回归解码器，生成任务的王者
3. **Vision Transformer**：将Transformer引入计算机视觉
4. **效率优化变体**：线性注意力、滑动窗口等

### 🌍 应用领域扩展：
- **自然语言处理**：从理解到生成的全面覆盖
- **计算机视觉**：图像分类、目标检测、分割
- **多模态学习**：图文理解、视频分析
- **科学计算**：蛋白质、材料、气候建模
- **其他领域**：语音、推荐、时间序列

### 🚀 技术发展趋势：
1. **效率优化**：降低计算复杂度，实现民主化部署
2. **多模态融合**：走向通用人工智能
3. **长序列建模**：处理更复杂的推理任务
4. **架构创新**：探索Transformer的替代方案

### 💡 关键洞察：
- **统一架构的力量**：同一套机制适用于多种任务和模态
- **规模化的效应**：更大的模型带来意想不到的能力涌现
- **注意力的通用性**：不仅仅是序列，任何关系建模都可以用
- **效率与性能的平衡**：实际应用需要在两者间找到最佳点

### 🔮 未来展望：
Transformer已经从一个机器翻译模型发展成为AI领域的基础架构。未来，我们可能看到：
- 更加高效的注意力机制
- 真正的多模态统一模型
- 在科学发现中的重大突破
- 向通用人工智能的进一步迈进

Transformer的故事还在继续，它的影响力将继续塑造AI的未来。通过这个完整的教程系列，你已经掌握了理解和应用这一革命性架构的所有基础知识。

### 🎓 学习完成！
恭喜你完成了完整的Transformer教程！现在你已经具备了：
- 深入理解Transformer的工作原理
- 实现各种Transformer变体的能力
- 将Transformer应用到不同领域的知识
- 对未来发展方向的前瞻性认识

继续探索，在AI的海洋中航行！🌊